<a href="https://colab.research.google.com/github/ImVis10/1PercentStrategy/blob/V1/1PercentStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv  
import re
import datetime
import csv
import requests

In [ ]:
API_KEY = 'OTPBXZ24V7Q9A24S'

In [ ]:
def write_to_csv(stock,data, mode):


  # header = ['time', 'open', 'high', 'low', 'close', 'volume']

  with open(stock+'_minute.csv', mode, encoding='UTF8') as f:
      writer = csv.writer(f)
      writer.writerow(data)

In [ ]:


def get_stock_data(stock_name):
  time_periods ={
    # '1':'year1month1',
    # '2':'year1month2',
    # '3':'year1month3',
    # '4':'year1month4',
    # '5':'year1month5',
    # '6':'year1month6',
    # '7':'year1month7',
    # '8':'year1month8',
    # '9':'year1month9',
    # '10':'year1month10',
    # '11':'year1month11',
    # '12':'year1month12',
    # '13':'year2month1',
    # '14':'year2month2',
    # '15':'year2month3',
    # '16':'year2month4',
    # '17':'year2month5',
    # '18':'year2month6',
    # '19':'year2month7',
    # '20':'year2month8',
    # '21':'year2month9',
    # '22':'year2month10',
    # '23':'year2month11',
    # '24':'year2month12',
}
  for month, time_period in time_periods.items():
    CSV_URL = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=%(stock)s&interval=1min&slice=%(slice)s&apikey=%(key)s&adjusted=false'\
            %{'stock':stock_name,'slice':time_period,'key':API_KEY}
    with requests.Session() as s:
      download = s.get(CSV_URL)
      decoded_content = download.content.decode('utf-8')
      cr = csv.reader(decoded_content.splitlines(), delimiter=',')
      my_list = list(cr)
      for row in my_list:
          write_to_csv('test', row,'a')
get_stock_data('IBM')          

In [ ]:
stock = 'AAPL'
mode = 'r'
interval = 'minute'
file_path = stock+'_' + interval +'.csv'
AAPL_minute_data = []
with open(file_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
            AAPL_minute_data.append(list(row))
print('done')

done


In [ ]:
# stock = 'IBM'
# mode = 'r'
# interval = 'daily'
# file_path = stock+'_' + interval +'.csv'
# IBM_daily_data = []
# with open(file_path) as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     for row in csv_reader:
#             IBM_daily_data.append(list(row))
# print('done')
print(AAPL_minute_data[0])

['12/1/2022 20:00', '147.85', '147.86', '147.82', '147.86', '1137']


In [ ]:
daily_close_price = []


In [ ]:
AAPL_minute_data.reverse()

In [ ]:

from datetime import timedelta
close_time_regex = "16:00$"
close_datetime = datetime.strptime( AAPL_minute_data[0][0], '%m/%d/%Y %H:%M')
# close_datetime = datetime.strptime( AAPL_minute_data[0][0][:9], '%m/%d/%Y ')
print(close_datetime)

counter = 0
for i in range(len(AAPL_minute_data)):
  current_datetime = AAPL_minute_data[i][0]
  # if counter == 100:
  #   break
  # print(close_datetime.date(), datetime.strptime( current_datetime, '%m/%d/%Y %H:%M').date())  
  # print (close_datetime.date()== datetime.strptime( current_datetime, '%m/%d/%Y %H:%M').date())
  # if re.search(close_time_regex, date_time ):
  # print(datetime.strptime( current_datetime, '%m/%d/%Y %H:%M'))
  if (close_datetime.date() == datetime.strptime( current_datetime, '%m/%d/%Y %H:%M').date() ):
    print('If', close_datetime.date(), datetime.strptime( current_datetime, '%m/%d/%Y %H:%M').date())
    daily_close_price.append(AAPL_minute_data[i])
    close_datetime = close_datetime - timedelta(days=1)
    # print(close_datetime)
  # else:
  #   print('Else', close_datetime.date(), datetime.strptime( current_datetime, '%m/%d/%Y %H:%M').date())
  counter+=1  



In [ ]:
date_and_time = AAPL_minute_data[0][0].split()
current_date = date_and_time[0]
for i in range(len(AAPL_minute_data)):
  # current_datetime = AAPL_minute_data[i][0]
  date_and_time = AAPL_minute_data[i][0].split()
  end_date = date_and_time[0]
  

  # if there is a change in the current_date, get the data
  if (current_date != end_date):

    # print('If', close_datetime.date(), datetime.strptime( current_datetime, '%m/%d/%Y %H:%M').date())
    # print('Current Date', current_date)
    daily_close_price.append(AAPL_minute_data[i-1])
    current_date = end_date
    # print(close_datetime)
  # else:
  #   print('Else', close_datetime.date(), datetime.strptime( current_datetime, '%m/%d/%Y %H:%M').date())
  counter+=1  

daily_close_price.append(AAPL_minute_data[-1])

In [ ]:
print(len(daily_close_price))
print(daily_close_price[0])


496
['12/14/2020 20:00', '121.89', '121.89', '121.89', '121.89', '895']


In [ ]:
AAPL_minute_data.reverse()

In [ ]:
daily_open_price = []

In [ ]:
# AAPL_minute_data.reverse()
open_time_regex = "../.*/.... ^9:31$"
counter = 0
for i in range(len(AAPL_minute_data)):
  date_time = AAPL_minute_data[i][0]
  if re.search(open_time_regex, date_time ):
    print(AAPL_minute_data[i])
    daily_open_price.append(AAPL_minute_data[i])
    counter+=1
  if counter == 10:
    break  
print(len(daily_open_price))    

0


In [ ]:
current_date = ''
for i in range(len(AAPL_minute_data)):
  # current_datetime = AAPL_minute_data[i][0]
  date_and_time = AAPL_minute_data[i][0].split()
  end_date = date_and_time[0]

  # if there is a change in the current_date, get the data
  if (current_date != end_date):

    # print('If', close_datetime.date(), datetime.strptime( current_datetime, '%m/%d/%Y %H:%M').date())
    # print('Current Date', current_date)
    daily_open_price.append(AAPL_minute_data[i])
    current_date = end_date
    # print(close_datetime)
  # else:
  #   print('Else', close_datetime.date(), datetime.strptime( current_datetime, '%m/%d/%Y %H:%M').date())
  counter+=1  


In [ ]:
# print(IBM_minute_data[69000:69400])
# Remove ['2021-11-26 09:31:00', ..] to clean data
print(len(daily_open_price))
print(daily_open_price[0])
print(daily_close_price[0])
# daily_open_price.pop(160)

496
['12/14/2020 4:01', '122.2', '122.3', '122.2', '122.26', '5379']
['12/14/2020 20:00', '121.89', '121.89', '121.89', '121.89', '895']


In [ ]:
#  Clean minute data to remove headers
# for i in range(len(IBM_minute_data)):
#   if IBM_minute_data[i][0] == '{':
#     # print(IBM_minute_data[i])
#     IBM_minute_data.pop(i)
#     # print(daily_open_price[i][0], "||",daily_close_price[i][0])



In [ ]:
counter = 0
# Clean daily data
for i in range(len(daily_close_price)):
  if daily_open_price[i][0][:10] != daily_close_price[i][0][:10]:
    daily_open_price.pop(i)
    # print(daily_open_price[i][0], "||",daily_close_price[i][0])
    counter+=1
# print(counter)


In [ ]:
# print(len(daily_open_price))
for i in range(len(daily_open_price)):
  print(daily_open_price[i][0],'||', daily_close_price[i][0])
  

In [ ]:
PRINCIPAL = 1000

In [ ]:
def close_trade(gain_or_loss):
  global PRINCIPAL
  PRINCIPAL += gain_or_loss


In [ ]:
# Principal
# Current_principal = Principal - buying_price
# Once stock_price meets desired profit(gain)
# Exit trade => Current_principal = current_principal + buying_price + Gain
########################### Go bull ##################

# Buy
# Wait until you make desired profit or hit the SL
# Exit
def go_bull(profit_percent,buy_price,time_series_data):
  print(time_series_data[0],'Timeseries dataaaaaaaaaaaaaaa yoooooooo')
  target = buy_price + (buy_price * profit_percent)
  stop_loss = profit_percent/4
  bottom_line = buy_price - (buy_price*stop_loss)
  print(buy_price,target,bottom_line)
  for row in time_series_data:
    # Check for stop loss first
    #  Comparing with open and low 
    if(max(float(row[1])  , float(row[3]))  <= bottom_line):
      print("SL Hit")
      print(buy_price,bottom_line,'max(float(row[1]), float(row[3]))', max(float(row[1]), float(row[3])))
      close_trade(max(float(row[1])  , float(row[3])) - buy_price)
      break
    # Comparing with open and high 
    if(float(row[1]) , float(row[2])  >= target):
      print("Profit Hit")
      print(buy_price,target,'max(float(row[1]), float(row[2]))', max(float(row[1]), float(row[2])))
      close_trade(max(float(row[1])  , float(row[2])) - buy_price)
      break
  close_trade(buy_price - float(time_series_data[-1][4]))   


In [ ]:
# Principal
# Current_principal = Principal - buying_price
# Once stock_price meets desired profit(gain)
# Exit trade => Current_principal = current_principal + buying_price + Gain
########################### Go bear ##################

# Buy
# Wait until you make desired profit or hit the SL
# Exit

def go_bear(profit_percent, sell_price, time_series_data):
  print(time_series_data[0],'Timeseries dataaaaaaaaaaaaaaa')
  target = sell_price - (sell_price * profit_percent)
  stop_loss = profit_percent/4
  top_line = sell_price + (sell_price*stop_loss)
  print(sell_price,target,top_line)

  for row in time_series_data:
    print(row)


  for row in time_series_data:
    # Check for stop loss first
    #  Comparing with open and high
    if(min(float(row[1]), float(row[2]))  >= top_line):
      print("SL Hit")
      print(sell_price,top_line,'min(float(row[1]), float(row[2]))', min(float(row[1]), float(row[2])))
      close_trade(sell_price - min(float(row[1])  , float(row[2])))
      break
    # Comparing with open and low 
    if(min(float(row[1]),float( row[3]))  <= target):
      print("Profit Hit")
      print(sell_price, target,'min(float(row[1]), float(row[3]))', min(float(row[1]), float(row[3])))
      close_trade(sell_price - min(float(row[1])  , float(row[3])))
      break
  # Neither conditions hit. Close at the end of the trading day 
  close_trade(sell_price - float(time_series_data[-1][4]))   

In [ ]:
def get_minute_data(date):

  minute_data = []
  print(date)

  # Column header should not be considered hence len - 1
  for i in range(1, len(AAPL_minute_data) - 1):
      current_date = AAPL_minute_data[i][0].split()[0]
      if (current_date == date):
        minute_data.append(AAPL_minute_data[i])
        next_date = AAPL_minute_data[i+1][0].split()[0]
        if (current_date != next_date):
          break
  minute_data   
  return minute_data  
print(get_minute_data(daily_close_price[0][0].split()[0]))  
# print(AAPL_minute_data[1][0])  

12/14/2020
[['12/14/2020 4:04', '122.3', '122.3', '122.21', '122.21', '325'], ['12/14/2020 4:05', '122.28', '122.28', '122.28', '122.28', '335'], ['12/14/2020 4:06', '122.35', '122.35', '122.35', '122.35', '902'], ['12/14/2020 4:07', '122.35', '122.35', '122.35', '122.35', '503'], ['12/14/2020 4:09', '122.36', '122.47', '122.36', '122.47', '973'], ['12/14/2020 4:12', '122.31', '122.31', '122.25', '122.25', '1666'], ['12/14/2020 4:13', '122.24', '122.24', '122.22', '122.22', '716'], ['12/14/2020 4:14', '122.2', '122.2', '122.2', '122.2', '264'], ['12/14/2020 4:15', '122.19', '122.23', '122.19', '122.23', '1201'], ['12/14/2020 4:16', '122.23', '122.23', '122.23', '122.23', '1525'], ['12/14/2020 4:19', '122.23', '122.23', '122.23', '122.23', '771'], ['12/14/2020 4:20', '122.23', '122.23', '122.23', '122.23', '1074'], ['12/14/2020 4:21', '122.23', '122.23', '122.23', '122.23', '427'], ['12/14/2020 4:22', '122.23', '122.23', '122.22', '122.23', '4711'], ['12/14/2020 4:25', '122.16', '122.16

In [ ]:
# def get_minute_data(date):
#   # date_timestamp = datetime.strptime(date, '%Y-%m-%d')
#   minute_data = []
#   date_and_time = AAPL_minute_data[i][0].split()
#   end_date = date_and_time[0]

#   # Column header should not be considered hence len - 1
#   for i in range( len(AAPL_minute_data)):
#     date_time_timestamp = datetime.strptime(AAPL_minute_data[i][0], format_string)
#     if re.search(date_regex, date_time ) and (date_time_timestamp > min_time and date_time_timestamp < max_time ):
#       minute_data.append(AAPL_minute_data[i])
#   minute_data.reverse()    
#   return minute_data  
# print(get_minute_data(daily_close_price[0][0][:10]))  

In [ ]:
print(daily_open_price[0])
print(daily_close_price[0])

['12/14/2020 4:01', '122.2', '122.3', '122.2', '122.26', '5379']
['12/14/2020 20:00', '121.89', '121.89', '121.89', '121.89', '895']


In [ ]:
PRINCIPAL = 1000
for i in range(len(daily_open_price)-1):
  
  
  print(i,i+1)
  prev_stock_price = float(daily_close_price[i][1])
  stock_price = float(daily_open_price[i+1][1])
  bullish_condition = stock_price > prev_stock_price * 1.01
  bearish_condition = stock_price < prev_stock_price * 0.99
  # print(stock_price,'|',prev_stock_price)
  # print(bullish_condition)
  # print(bearish_condition)

  if(bullish_condition):
    # go_bull
    time_series = get_minute_data(daily_open_price[0][0].split()[0])
    print("Going bull",time_series)
    go_bull(0.01, stock_price,time_series)
    None
  elif(bearish_condition):
    print("Going bear")
    time_series = get_minute_data(daily_open_price[0][0].split()[0])
    go_bull(0.01, stock_price,time_series)

    # go bear

    # Exec Trade


0 1
1 2
2 3
3 4
4 5
5 6
6 7
12/14/2020
Going bull [['12/14/2020 4:04', '122.3', '122.3', '122.21', '122.21', '325'], ['12/14/2020 4:05', '122.28', '122.28', '122.28', '122.28', '335'], ['12/14/2020 4:06', '122.35', '122.35', '122.35', '122.35', '902'], ['12/14/2020 4:07', '122.35', '122.35', '122.35', '122.35', '503'], ['12/14/2020 4:09', '122.36', '122.47', '122.36', '122.47', '973'], ['12/14/2020 4:12', '122.31', '122.31', '122.25', '122.25', '1666'], ['12/14/2020 4:13', '122.24', '122.24', '122.22', '122.22', '716'], ['12/14/2020 4:14', '122.2', '122.2', '122.2', '122.2', '264'], ['12/14/2020 4:15', '122.19', '122.23', '122.19', '122.23', '1201'], ['12/14/2020 4:16', '122.23', '122.23', '122.23', '122.23', '1525'], ['12/14/2020 4:19', '122.23', '122.23', '122.23', '122.23', '771'], ['12/14/2020 4:20', '122.23', '122.23', '122.23', '122.23', '1074'], ['12/14/2020 4:21', '122.23', '122.23', '122.23', '122.23', '427'], ['12/14/2020 4:22', '122.23', '122.23', '122.22', '122.23', '4711']

In [ ]:
print(PRINCIPAL)

1014.3499999999998


In [ ]:
from datetime import datetime
datetime_str = '2022-10-10 09:30:00'
date_str = '2022-10-10'
now = datetime.strptime(date_str, '%Y-%m-%d')
timestamp = datetime.timestamp(now)
print("timestamp =", timestamp)


timestamp = 1665360000.0
